# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
full_data_rows_list = [] 
    
for f in file_path_list:
 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:  
        csvreader = csv.reader(csvfile) 
        next(csvreader)
               
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [ ]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()
print('Connected')

#### Create Keyspace

In [ ]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS project2
         WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
try:
    session.set_keyspace('project2')
    print('KeySpace set')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### QUERY 1. 
**Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4**

To obtain a result from this query we need to create a table called session_library that contains the variables session_id, item_in_session, artist, song and length. The session_id is the the primary key used for partitioning the data. The data is clusterized by item_in_session. item_in_session has repeated values, so it will generate enough clusters and can be performant as well when querying the table.

Once the table is created we will insert values into this table and execute the query.



In [ ]:
table_1 = "CREATE TABLE IF NOT EXISTS session_library"
table_1 = table_1 + ("(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY(session_id, item_in_session))")

try:
    session.execute(table_1)
except Exception as e:
    print(e)

                    

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO session_library(session_id, item_in_session, artist, song, length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
 
        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
table_1_query = """
    SELECT artist, song, length FROM session_library
    WHERE session_id = %s AND item_in_session = %s
"""


try:
    rows = session.execute(table_1_query, (338, 4))
except Exception as e:
    print(e)
for row in rows:
    print(row.artist, row.song, row.length)



### Query 2

** Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 **

We will create a table called user_sessions, containing the variables session_id, user_id, artist, song, first_name, last_name and item_in_session. Both first_name and last_name are included to create the user info. The session_id and user_id are both partitioning keys. This is to avoid that session_id's tied to the user_id will be distributed over multiple nodes. When this happens, there will be more performance issues once the data scales. item_in_session is the clustering key and is added to order the records in the table.

Once the table is created we will insert values into this table and execute the query.



In [ ]:
table_2 = "CREATE TABLE IF NOT EXISTS user_sessions"
table_2 = table_2 + "(session_id int, user_id int, artist text, song text, first_name text, last_name text, item_in_session int, PRIMARY KEY((session_id, user_id), item_in_session)) "

try:
    session.execute(table_2)
except Exception as e:
    print(e)
                    

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_sessions(session_id, user_id, artist, song, first_name, last_name, item_in_session)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
 
        session.execute(query, (int(line[8]), int(line[10]), str(line[0]), str(line[9]), str(line[1]), str(line[4]), int(line[3])))

In [ ]:
table_2_query = """
    SELECT artist, song, first_name, last_name FROM user_sessions
    WHERE user_id = %s AND session_id = %s
"""
try:
    rows = session.execute(table_2_query, (10, 182))
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)


### Query 3

** Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own' **

We will create a table called listener_history that contains the variables song, first_name, last_name, user_id. Song is the partioning key, but we need a clustering key user_id. Without the user_id as clustering key the users listening to the same song would get overwritten consistently. Song is not unique and therefore it needs the user_id as clustering key.

Once the table is created we will insert values into this table and execute the query.

In [ ]:
table_3 = "CREATE TABLE IF NOT EXISTS listener_history"
table_3 = table_3 + ("(song text, first_name text, last_name text, user_id int, PRIMARY KEY(song, user_id))")

try:
    session.execute(table_3)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO listener_history(song, first_name, last_name, user_id)"
        query = query + "VALUES(%s, %s, %s, %s)"

        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [ ]:
table_3_query = """
    SELECT first_name, last_name FROM listener_history WHERE song = 'All Hands Against His Own'
"""

try:
    rows = session.execute(table_3_query)
except Exception as e:
    print(e)

for row in rows:
    print(row.first_name, row.last_name)

### Drop the tables before closing out the sessions

In [ ]:
session.execute('DROP TABLE IF EXISTS session_library')
session.execute('DROP TABLE IF EXISTS user_sessions')
session.execute('DROP TABLE IF EXISTS listener_history')

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()